In [81]:
#Adapted from code supplements from papers: 
#1.`Modewise Operators, the Tensor Restricted Isometry Property, and Low-Rank Tensor Recovery" by Mark A. Iwen, Deanna Needell, Michael Perlmutter, Elizaveta Rebrova
#
#2. ``Iterative Hard Thresholding for Low CP-rank Tensor Models" by R. Grotheer, S. Li, A. Ma, D. Needell, J. Qi
#

#Imports
import torch
import torch.nn.functional as F

import tensorly as tl
from tensorly import decomposition
from tensorly import random
from tensorly.decomposition import parafac

import numpy as np
from scipy.stats import ortho_group
from scipy import linalg
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from PIL import Image

from scipy.linalg import dft

import timeit

### TO D0: How to include if error??

In [82]:
#Returns low CP rank r tensor of dimension n

def random_low_orth_rank(n,r): ### Orthogonal Rank r  ##Errors in this function too
    #torch.manual_seed(0)
    #np.random.seed(0)
    L = []
    C= ortho_group.rvs((n[0],r))
    L = L + [C]
    
    for i in range(1,len(n)):
        C=np.random.normal(0,1,size=(n[i],r))
        L = L + [C]
    
    X = np.zeros(n)
    for i in range(r):
        U_r = np.array(L[0])[:,i]
        for j in range(1, len(n)):
            prod = np.array(L[j])[:,i]
            U_r = np.multiply.outer(U_r,prod)
        X = X + U_r
        
    C=tl.tensor(X) #Changing data frame to tensor
    C.shape
    return C

def random_low_cp_rank(n,r):   #### CP Rank r
    #torch.manual_seed(0)
    #np.random.seed(0)
    
    L = []
    for i in range(0,len(n)):
        C = np.random.normal(0,1,size=(n[i],r))
        L = L + [C]
    
    X = np.zeros(n)
    for i in range(r):
        U_r = np.array(L[0])[:,i]
        for j in range(1, len(n)):
            prod = np.array(L[j])[:,i]
            U_r = np.multiply.outer(U_r,prod)
        X = X + U_r
        
    C=tl.tensor(X) #Changing data frame to tensor
    C.shape
    return C


### Some errors in this function, why?
def random_low_cp_rank_1(n,r): ### Issues with printing; also, source of randomness?
    tensor_1 = tl.random.random_cp(shape=n, rank=r)
    return tensor_1

In [83]:
def vectorize_1(X): ##Vectorisation for tensors
    x=X.numpy()
    x=x.reshape(-1)
    return x

def vectorize(X):  ##Vecorisation for numpy arrays
    x=X
    x=x.reshape(-1)
    return x

In [84]:
#Low rank approximation via tensorly Cp decomposition is used for thresholding

def low_rank_approx(tensor,r):    
    #torch.manual_seed(0)
    factors = parafac(tl.tensor(tensor), rank=r)
    answer = tl.cp_to_tensor(factors)
    return answer

In [120]:
#Measurement operators 

#SJLT
def SJLT(n, m, s, X):
    
    # It seems that in numpy, the function np.random.choice(m, s, replace=False) is the only one to perform uniform sampling without replacement. Unfortunately, this function does not support vectorization, and hence is highly inefficient. Hence in our implementation of SJLT, to speed-up SJLT, the function of sampling without replacement is implemented based on an early stopping of Fisher-Yates shuffle.
    d = X.shape[1]; random_signs = np.zeros((n, s))
    for i in range(n): 
        for j in range(s): 
            random_signs[i, j] = np.random.randint(2) * 2 - 1
    out = np.zeros((m, X.shape[1]))
    tmp_rands = np.random.rand(s * n)
    rand_numbers = np.zeros((s, n), dtype = np.int64)
    tmp_m_sequence = np.arange(m, dtype = np.int64)
    for j in range(n):
        for k in range(s):
            tmp_index = int(tmp_rands[ j * s + k ] * (m-k))
            tmp = tmp_m_sequence[tmp_index + k]
            tmp_m_sequence[tmp_index + k] = tmp_m_sequence[k]
            tmp_m_sequence[k] = tmp
            rand_numbers[k,j] = tmp
    
    for j in range(n): 
        for k in range(s): out[rand_numbers[k, j], :] += random_signs[j,k] * X[j, :]
    return out / math.sqrt(s)


#Gaussian
def create_gaussian_meas(dim, n):
    #np.random.seed(0)
    return np.sqrt(1/dim)*np.random.normal(0.0, 1.0, [dim, n])

def create_khatri_rao_gaussian(dim,n):
    a = np.random.normal(0.0, 1.0, [n[0], dim])
    
    matrix = a
    
    for j in range(1,len(n)):
        a = np.random.normal(0.0, 1.0, [n[j], dim])
        matrix = linalg.khatri_rao(matrix, a)
    
    return matrix

#SORS
def create_SORS_meas(dim,n):
    if n<dim:
        raise ValueError("dim is greater than n, matrix needs to be tall and skinny")
    
    n_prod = 1
    for i in n:
        n_prod = n_prod*i   
        
    #np.random.seed(0)
    m=dft(n_prod)/np.sqrt(n_prod)
    vec=np.random.choice([-1,1],n_prod)
    m = np.matmul(m, np.diag(vec))
    m = np.sqrt(n_prod/dim)*m[:int(dim), :]
    return m  

In [87]:
np.shape(create_khatri_rao_gaussian(10,(3,3,2)))

(18, 10)

In [88]:
##Generate Random Tensors
def generate_cp_tensors(n,r,num_samples):

    #torch.manual_seed(0)
    X= []
    for j in range(num_samples):
        X_i= torch.tensor(random_low_cp_rank(n,r), dtype=torch.float64)
        X.append(X_i)
        
    return X

##Generate Random Tensors orthogonal rank
def generate_orth_tensors(n,r,num_samples):

    #torch.manual_seed(0)
    X= []
    for j in range(num_samples):
        X_i= torch.tensor(random_low_orth_rank(n,r), dtype=torch.float64)
        X.append(X_i)
        
    return X

In [89]:
#Low rank approximation via tensorly Cp decomposition is used for thresholding

def low_rank_approx(tensor,r):    
    #torch.manual_seed(0)
    factors = parafac(tl.tensor(tensor), rank=r)
    answer = tl.cp_to_tensor(factors)
    return answer

In [90]:
## TIHT for one sample!

def CP_TIHT_1(X, n, AA, r, yy, mu = 1, acc = 0.001, num_its = 1000):
    
    vXX = torch.randn(n)
    j = 1
    try:
    
        while np.linalg.norm(vectorize(vXX)- vectorize_1(X)) > acc and j < num_its:
            WW = np.array(vectorize(vXX)) + mu * np.matmul(AA.T, (yy - np.matmul(AA, np.array(vectorize(vXX)))))
            WW = torch.reshape(torch.tensor(WW), n)
            vXX = low_rank_approx(WW,r)
            #print(np.linalg.norm(vectorize(vXX)- vectorize_1(X)),j)
            j = j+1 
    
    except np.linalg.LinAlgError as err:
        j = num_its
    
    #print(np.linalg.norm(vectorize(vXX)- vectorize_1(X)))
    return np.linalg.norm(vectorize(vXX)- vectorize_1(X)), j, vXX

In [91]:
## Testing Function
X = generate_cp_tensors((10,11,10),2,1)
n = [10,11,10]
n_prod = 10*11*10
X= X[0]

In [92]:
mu = 1
dim = 500
AA = np.sqrt(1/dim)*np.random.normal(0.0, 1.0, [dim, n_prod])
yy = np.matmul(AA, np.array(vectorize(X)))

In [93]:
## Question: How to deal with errors, add 0 to code?

x, y, vXX = CP_TIHT_1(X, n, AA, 2, yy, mu = 1, acc = 0.001, num_its = 1000)

In [94]:
print("Iterations :", y, " Frob norm :", x)

Iterations : 26  Frob norm : 0.000884017401804273


In [14]:
### TIHT for more samples
## Input:
## Output: 

def CP_TIHT(n,r,dim,num_samples=100,mu=1,num_its=1000,accuracy=.001, meas = "Gaussian"):

    #torch.manual_seed(0)

    X = generate_cp_tensors(n,r,num_samples) ## generating num_samples tensors
    
    n_prod = 1
    for i in n:
        n_prod = n_prod*i 
        
    AA = create_gaussian_meas(dim,n_prod)
    
    good_runs = 0
    total_time = 0
    total_iters = 0
    
    iterations = []
    norm = []

    vXX = torch.randn(n) ##Initialisation

    # Run recovery algorithm
    for j in range(num_samples):
        #print(j)
        
        yy = np.matmul(AA, np.array(vectorize(X[j])))
        start = timeit.default_timer()
        norm_1, its_1, test = CP_TIHT_1(X[j], n, AA, r, yy, mu, accuracy, num_its)
        #print(norm_1)
        stop = timeit.default_timer()
    
        #plt.plot(range(len(Losses3[j])), Losses3[j])   
        if its_1 < num_its:
            good_runs += 1
            total_time += stop - start
            total_iters += its_1
            #print("Converged!")
            #print('Number of iterations: ', i)
            
        iterations = iterations + [its_1]
        norm = norm + [norm_1]
    '''if good_runs != 0:
        print('\n')
        print('Percentage of converged runs:', 100*good_runs/num_samples)
        print('Average recovery time: ', total_time/good_runs) 
        print('Average number of iterations: ', total_iters/good_runs) 
    else:
        print("Never converged :(")'''
        
    if good_runs != 0:
        Convergence_percent=100*good_runs/num_samples
        Average_recovery_time= total_time/good_runs
        Average_number_of_iterations= total_iters/good_runs 
   
    else:
        Convergence_percent=0
        Average_recovery_time= np.inf
        Average_number_of_iterations= num_its
   
    return Convergence_percent, Average_recovery_time, Average_number_of_iterations, norm, iterations

In [27]:
cp, rt, ai, norm, its = CP_TIHT(n,2,400,5,mu=1,num_its=1000,accuracy=.001)

In [28]:
## Testing Function
## cp

100.0

In [29]:
######## Gaussian Measurements #######
######## Fixed dimensions (10,10,10) changing rank #################

In [30]:
dim = [800,750,700,650,600,550,500,450,400,350,300]
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT((10,10,10),2,dim[i],100,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]
    

Dimension : 800  Convergence percent : 100.0  Avg_its : 16.51
Dimension : 750  Convergence percent : 100.0  Avg_its : 17.5
Dimension : 700  Convergence percent : 100.0  Avg_its : 18.58
Dimension : 650  Convergence percent : 100.0  Avg_its : 20.51
Dimension : 600  Convergence percent : 100.0  Avg_its : 21.56
Dimension : 550  Convergence percent : 100.0  Avg_its : 24.87
Dimension : 500  Convergence percent : 100.0  Avg_its : 28.69
Dimension : 450  Convergence percent : 100.0  Avg_its : 36.4
Dimension : 400  Convergence percent : 100.0  Avg_its : 54.3
Dimension : 350  Convergence percent : 88.0  Avg_its : 119.25
Dimension : 300  Convergence percent : 43.0  Avg_its : 198.6046511627907


In [63]:
dim = [800,750,700,650,600,550,500]
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT((10,10,10),3,dim[i],100,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

Dimension : 800  Convergence percent : 100.0  Avg_its : 28.29
Dimension : 750  Convergence percent : 100.0  Avg_its : 29.87
Dimension : 700  Convergence percent : 100.0  Avg_its : 35.14
Dimension : 650  Convergence percent : 100.0  Avg_its : 43.38
Dimension : 600  Convergence percent : 99.0  Avg_its : 60.313131313131315
Dimension : 550  Convergence percent : 99.0  Avg_its : 84.03030303030303
Dimension : 500  Convergence percent : 89.0  Avg_its : 171.96629213483146


In [65]:
dim = [800,750,700,650]
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT((10,10,10),4,dim[i],100,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

Dimension : 800  Convergence percent : 100.0  Avg_its : 58.19
Dimension : 750  Convergence percent : 99.0  Avg_its : 77.36363636363636
Dimension : 700  Convergence percent : 92.0  Avg_its : 155.95652173913044
Dimension : 650  Convergence percent : 76.0  Avg_its : 233.22368421052633


In [ ]:
dim = [800,750,700,650]
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT((10,10,10),5,dim[i],100,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

In [44]:
########## Measuring average error across iterations for TIHT across rank##################

def CP_TIHT_error_1(X, n, AA, r, yy,  num_its, mu = 1):
    
    vXX = torch.randn(n)
    error = []
    j = 1
    for i in range(len(num_its)):
        try:
    
            while j < num_its[i]:
                WW = np.array(vectorize(vXX)) + mu * np.matmul(AA.T, (yy - np.matmul(AA, np.array(vectorize(vXX)))))
                WW = torch.reshape(torch.tensor(WW), n)
                vXX = low_rank_approx(WW,r)
                #print(np.linalg.norm(vectorize(vXX)- vectorize_1(X)),j)
                j = j+1 
    
        except np.linalg.LinAlgError as err:
            j = num_its
        error = error + [np.linalg.norm(vectorize(vXX)- vectorize_1(X))/np.linalg.norm(vectorize_1(X))]
        
    return error

### TIHT for more samples
## Input:
## Output: 


def CP_TIHT_error(n,r,dim,num_samples=100,mu=1,num_its=[20,60,120,200], meas = "Gaussian"):

    #torch.manual_seed(0)

    X = generate_cp_tensors(n,r,num_samples) ## generating num_samples tensors
    
    n_prod = 1
    for i in n:
        n_prod = n_prod*i 
        
    AA = create_gaussian_meas(dim,n_prod)
    
    cumulative_error = np.zeros(len(num_its))
    

    # Run recovery algorithm
    for j in range(num_samples):
        
        yy = np.matmul(AA, np.array(vectorize(X[j])))
        start = timeit.default_timer()
        err_j = CP_TIHT_error_1(X[j], n, AA, r, yy, num_its, mu)

        for k in range(len(num_its)):
            cumulative_error[k] += err_j[k]
    
 
    return cumulative_error


In [57]:
dim_error = []
for i in [800,700,600,500,400]:
    dim_error = dim_error + [CP_TIHT_error((10,10,10),3,i,10,mu=1)/10]
    print(i)

800
700
600
500
400


In [58]:
dim_error

[array([2.46177102e-04, 3.76247800e-07, 1.23839785e-08, 1.21975339e-08,
        1.24928665e-08, 1.04873670e-08, 8.48554913e-09, 1.03464644e-08,
        1.06059595e-08]),
 array([1.21952906e-03, 1.57957151e-05, 2.91154322e-07, 1.49414732e-08,
        9.86503613e-09, 9.64600333e-09, 7.64252551e-09, 9.44060689e-09,
        8.20087358e-09]),
 array([6.77528285e-03, 4.24841701e-04, 3.14062661e-05, 2.50342636e-06,
        2.14655639e-07, 2.86923488e-08, 1.21791166e-08, 1.09589762e-08,
        1.15866463e-08]),
 array([1.01918166e-01, 1.01953318e-01, 1.50624969e-01, 4.06761777e-01,
        3.34662772e+00, 5.38772823e+01, 1.19790344e+03, 1.30543133e+07,
        2.11883191e+22]),
 array([6.74843330e+00, 4.03534092e+03, 6.19278626e+06, 1.92263081e+10,
        9.96805465e+13, 6.37743306e+17, 4.34047920e+21, 1.45527022e+33,
        3.91899177e+71])]

In [60]:
dim_error = []
for i in [800,700,600,500,400]:
    dim_error = dim_error + [CP_TIHT_error((10,10,10),2,i,10,mu=1)/10]
    print(i)

800
700
600
500
400


In [61]:
dim_error

[array([6.53808331e-06, 4.94166806e-09, 4.44558610e-09, 4.08884529e-09,
        6.05753140e-09, 4.91592787e-09, 5.02737430e-09, 4.07994714e-09,
        5.63713211e-09]),
 array([1.54774550e-05, 1.31048029e-08, 7.61348777e-09, 7.07431860e-09,
        1.01060094e-08, 1.05073100e-08, 8.45187105e-09, 9.51460336e-09,
        1.01391647e-08]),
 array([9.05278502e-05, 9.93123480e-08, 5.87784248e-09, 3.87621477e-09,
        5.10573130e-09, 4.73308406e-09, 4.60578606e-09, 3.01166894e-09,
        4.66231412e-09]),
 array([1.06641061e-03, 1.90698786e-05, 4.01144313e-07, 1.51771219e-08,
        8.81657399e-09, 7.59918800e-09, 7.60798851e-09, 7.63620599e-09,
        7.49677474e-09]),
 array([8.49714164e-03, 1.37139779e-03, 2.55758654e-04, 4.88375246e-05,
        9.41749051e-06, 1.83228866e-06, 3.61711141e-07, 7.21074298e-09,
        3.93179578e-09])]

In [147]:
######## Gaussian Measurements #######
######## Fixed Rank changing Dimension n #################

import math
percent = [0.9,0.85,0.8,0.75,0.7,0.65,0.60]

In [156]:
n = [10,10,10]
n_prod = 1

for i in range(len(n)):
    n_prod = n_prod*n[i]
    
dim = []
percent = [0.8,0.6,0.4, 0.3]

for i in range(len(percent)):
    dim = dim + [math.ceil(n_prod*percent[i])]
    
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT(n,3,dim[i],50,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

Dimension : 800  Convergence percent : 100.0  Avg_its : 26.88
Dimension : 600  Convergence percent : 100.0  Avg_its : 57.04
Dimension : 400  Convergence percent : 30.0  Avg_its : 924.1333333333333


KeyboardInterrupt: 

In [154]:
n = [20,20,20]
n_prod = 1

for i in range(len(n)):
    n_prod = n_prod*n[i]
    
dim = []

for i in range(len(percent)):
    dim = dim + [math.ceil(n_prod*percent[i])]
    
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT(n,3,dim[i],50,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

Dimension : 6400  Convergence percent : 100.0  Avg_its : 10.9
Dimension : 4800  Convergence percent : 100.0  Avg_its : 12.26
Dimension : 3200  Convergence percent : 100.0  Avg_its : 15.62
Dimension : 2400  Convergence percent : 100.0  Avg_its : 19.7


In [155]:
n = [30,30,30]
n_prod = 1

for i in range(len(n)):
    n_prod = n_prod*n[i]
    
dim = []

for i in range(len(percent)):
    dim = dim + [math.ceil(n_prod*percent[i])]
    
conv_per = []
avg_its = []

for i in range(len(dim)):
    cp, rt, ai, norm, its = CP_TIHT(n,3,dim[i],5,mu=1,num_its=1000,accuracy=.001)
    print("Dimension :", dim[i], " Convergence percent :", cp, " Avg_its :", ai)
    conv_per = conv_per + [cp]
    avg_its = avg_its + [ai]

Dimension : 21600  Convergence percent : 100.0  Avg_its : 9.0
Dimension : 16200  Convergence percent : 100.0  Avg_its : 9.4
Dimension : 10800  Convergence percent : 100.0  Avg_its : 11.0
Dimension : 8100  Convergence percent : 100.0  Avg_its : 12.0
